In [1]:
#TODO 

# Install Packages

In [2]:
%pip install sdv==1.0.0
# Restart runtime after running this

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%pip install exrex --quiet

# Load Data

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('./input_for_CTGAN.csv')
data = data.sample(frac=1).reset_index(drop=True) #shuffle

cols = ['Unnamed: 0.1','Unnamed: 0']
for col in cols:
    if col in data.columns:
        data = data.drop(col, axis=1)

print(data.shape)
data.head(4)

(863, 15)


,Claim No,Card No,Diagnosis,Surgery/Therapy,Provider Name,Date of Admit,Status,Status Date,Est Amt,Pre Auth Appr Amt,Surgery/Therapy Date,Discharged,Length of Stay,Treatment Duration,Claim Duration
0,APCMCO/ANT/2022/2/12246524,WAP124601200443/03,CAD ACS,Management Of Acute MI With Angiogram,SAVEERA INSTITUTE OF MEDICAL SCIENCES PVT LTD,2022-04-23,Claim Paid,2022-06-20,30785.0,25785.0,2022-04-23,2022-04-25,2.0,0.0,58.0
1,APCMCO/ANT/2022/2/12258840,WAP1256017A0133/01,CAD - IHD,Management Of Acute MI With Angiogram,CHANDRA SUPERSPECIALTY HOSPITAL,2022-06-09,Claim Paid,2022-08-19,30785.0,30785.0,2022-06-11,2022-06-13,4.0,2.0,71.0
2,APCMCO/ANT/2022/2/12270278,WAP123001200212/02,HEART FAILURE,Medical Management of Refractory Cardiac Failure,SAVEERA INSTITUTE OF MEDICAL SCIENCES PVT LTD,2022-07-26,CEO Claim Approved,2022-08-27,41048.0,30548.0,2022-07-26,2022-07-28,2.0,0.0,32.0
3,APCMCO/ANT/2022/2/12242535,WAP124700600421/01,AWMI,Management Of Acute MI With Angiogram,SAVEERA INSTITUTE OF MEDICAL SCIENCES PVT LTD,2022-04-08,Claim Paid,2022-06-20,30785.0,20785.0,2022-04-09,2022-04-11,3.0,1.0,73.0


# Data preprocessing

In [6]:
# Dropping computable columns
data = data.drop(['Length of Stay','Treatment Duration','Claim Duration'], axis=1)

# dropping duplicate rows
data = data.drop_duplicates()

# dropping null valued est amt; these are duplicate rows. 
data = data.dropna(subset=['Est Amt'])
data = data.reset_index(drop=True)

print("All Claim No are unqiue." if data['Claim No'].is_unique else "All Claim No are not unique.")
data.shape

All Claim No are unqiue.


(853, 12)

In [7]:
import warnings

def changeDTypes(df, cols, toTypes):
    #TODO: both 'int' and 'float' are converted to just numericals; have to work with NaN.
    for col, totype in zip(cols, toTypes):
        if col in df.columns:
            if totype == 'int':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            elif totype == 'float':
                df[col] = pd.to_numeric(df[col], errors='coerce')
            elif totype == 'datetime':
                df[col] = pd.to_datetime(df[col], dayfirst=True) #TODO: without time
            elif totype == 'string':
                df[col] = df[col].astype(str)
        else:
            warnings.warn( f'"{col}" column not present in dataframe.')
    return df

In [8]:
cols = ['Claim No','Card No', 'Diagnosis','Surgery/Therapy', 'Provider Name','Date of Admit', 'Status', 'Status Date', 'Est Amt','Pre Auth Appr Amt', 'Surgery/Therapy Date', 'Discharged']
dtypes = ['str', 'str', 'str', 'str', 'str', 'datetime', 'str','datetime', 'float', 'float', 'datetime', 'datetime']
data = changeDTypes(data, cols, dtypes)

In [9]:
data.columns

Index(['Claim No', 'Card No', 'Diagnosis', 'Surgery/Therapy', 'Provider Name',
       'Date of Admit', 'Status', 'Status Date', 'Est Amt',
       'Pre Auth Appr Amt', 'Surgery/Therapy Date', 'Discharged'],
      dtype='object')

In [10]:
# df = data.drop(['Claim No'],axis=1)
df = data.copy()
df = df[~data['Card No'].str.startswith('CMCO/APS')] #just 2 observations. to make later generation part easy

## Replace the Card-ID
else generator will generate real card ISs

In [11]:
#generate string from regex pattern
import exrex #some issue with licensing. #TODOto check
card_pattern = r'^(APCMCO|APTRUST)/(ANT)/(2022)/(1|2)/\d{8}(/01|/02)?'

#create one-to-one map of Card No; then substitute; to preserve the distribution
real_fake_map = {}
for real_card in df['Card No'].unique():
    real_fake_map[real_card] = exrex.getone(card_pattern)
df['Card No'] = df['Card No'].apply(lambda x: real_fake_map[x])


# df[~df['Claim No'].str.contains(pattern)]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 851 entries, 0 to 852
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Claim No              851 non-null    object        
 1   Card No               851 non-null    object        
 2   Diagnosis             851 non-null    object        
 3   Surgery/Therapy       851 non-null    object        
 4   Provider Name         851 non-null    object        
 5   Date of Admit         851 non-null    datetime64[ns]
 6   Status                851 non-null    object        
 7   Status Date           851 non-null    datetime64[ns]
 8   Est Amt               851 non-null    float64       
 9   Pre Auth Appr Amt     851 non-null    float64       
 10  Surgery/Therapy Date  851 non-null    datetime64[ns]
 11  Discharged            820 non-null    datetime64[ns]
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 86.4+ KB


# Metadata Preparation

In [13]:
# Metadata preparation
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df)

In [14]:
metadata.update_column(
    column_name='Claim No',
    sdtype='id',
)
metadata.set_primary_key(column_name='Claim No')

In [15]:
metadata.to_dict()

{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'Claim No': {'sdtype': 'id'},
  'Card No': {'sdtype': 'categorical'},
  'Diagnosis': {'sdtype': 'categorical'},
  'Surgery/Therapy': {'sdtype': 'categorical'},
  'Provider Name': {'sdtype': 'categorical'},
  'Date of Admit': {'sdtype': 'datetime'},
  'Status': {'sdtype': 'categorical'},
  'Status Date': {'sdtype': 'datetime'},
  'Est Amt': {'sdtype': 'numerical'},
  'Pre Auth Appr Amt': {'sdtype': 'numerical'},
  'Surgery/Therapy Date': {'sdtype': 'datetime'},
  'Discharged': {'sdtype': 'datetime'}},
 'primary_key': 'Claim No'}

In [16]:
metadata.validate()

# Synthesizer

In [24]:
import torch
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=True, #decimal places as per input data
    epochs=5000,
    verbose=True,
    cuda=True,
    batch_size=516,   #needs to be divisible by pac
    pac=12,   #pack size to send to discriminator
    # discriminator_dim=(128,128),  #discriminator layers
    # discriminator_decay=0.0000001, #Adam optimizer decay
    # discriminator_lr = 0.001, #Learning rate
    # discriminator_steps= 2, #number of discriminator update for each generator update
    embedding_dim = 50, #size of the noise vector sent to generator
    # generator_decay = 1e-6, #generator weight decay for Adam optimizer
    generator_dim = (256,256,128,64), #A Residual Layer will be created for each one of the values provided
    # generator_lr = 0.001,
    # log_frequency =True, #use log frequency of categorical levels in conditional sampling #TODO
    
)

## Add constraints
treatment durations are negative also.

In [25]:
import sdv.constraints as SC

# Keep as less as possible; let GAN learn the real distribution
myConstraints = [
     {
        'constraint_class': 'Positive',
        'constraint_parameters': {
            'column_name': 'Est Amt',
            'strict_boundaries': False
        }
    },
    {
        'constraint_class': 'Positive',
        'constraint_parameters': {
            'column_name': 'Pre Auth Appr Amt',
            'strict_boundaries': False
        }
    }
]

synthesizer.add_constraints(constraints=myConstraints)

# Training CTGAN

In [26]:
%%time

synthesizer.fit(df)

Streaming output truncated to the last 5000 lines.
Epoch 3, Loss G:  4.1840,Loss D: -0.0342
Epoch 4, Loss G:  4.2403,Loss D: -0.0608
Epoch 5, Loss G:  4.1439,Loss D: -0.0943
Epoch 6, Loss G:  4.0359,Loss D: -0.1152
Epoch 7, Loss G:  4.0932,Loss D: -0.1409
Epoch 8, Loss G:  4.1317,Loss D: -0.2160
Epoch 9, Loss G:  4.1476,Loss D: -0.2350
Epoch 10, Loss G:  4.0737,Loss D: -0.2853
Epoch 11, Loss G:  4.0067,Loss D: -0.2917
Epoch 12, Loss G:  4.1410,Loss D: -0.2756
Epoch 13, Loss G:  4.0410,Loss D: -0.3180
Epoch 14, Loss G:  3.7721,Loss D: -0.2879
Epoch 15, Loss G:  3.9954,Loss D: -0.3313
Epoch 16, Loss G:  4.0798,Loss D: -0.3403
Epoch 17, Loss G:  3.9602,Loss D: -0.3322
Epoch 18, Loss G:  3.9664,Loss D: -0.3471
Epoch 19, Loss G:  4.1159,Loss D: -0.3349
Epoch 20, Loss G:  3.9291,Loss D: -0.3320
Epoch 21, Loss G:  4.1164,Loss D: -0.3670
Epoch 22, Loss G:  3.9320,Loss D: -0.4333
Epoch 23, Loss G:  4.0296,Loss D: -0.4021
Epoch 24, Loss G:  4.0462,Loss D: -0.4830
Epoch 25, Loss G:  3.9868,Loss D

# Models Architecture
Access to discriminator is not yet available.

In [27]:
type(synthesizer)

sdv.single_table.ctgan.CTGANSynthesizer

In [28]:
synthesizer._model._generator

Generator(
  (seq): Sequential(
    (0): Residual(
      (fc): Linear(in_features=1071, out_features=256, bias=True)
      (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): Residual(
      (fc): Linear(in_features=1327, out_features=256, bias=True)
      (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): Residual(
      (fc): Linear(in_features=1583, out_features=128, bias=True)
      (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (3): Residual(
      (fc): Linear(in_features=1711, out_features=64, bias=True)
      (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (4): Linear(in_features=1775, out_features=1060, bias=True)
  )
)

In [29]:
synthesizer.get_parameters()

{'enforce_min_max_values': True,
 'enforce_rounding': True,
 'embedding_dim': 50,
 'generator_dim': (256, 256, 128, 64),
 'discriminator_dim': (256, 256),
 'generator_lr': 0.0002,
 'generator_decay': 1e-06,
 'discriminator_lr': 0.0002,
 'discriminator_decay': 1e-06,
 'batch_size': 516,
 'discriminator_steps': 1,
 'log_frequency': True,
 'verbose': True,
 'epochs': 5000,
 'pac': 12,
 'cuda': True}

# Generating Data

In [30]:
import exrex

synthetic_data = synthesizer.sample(num_rows=1000)

# Set Claim as generated
#Actual pattern r'^(((JAP|WAP|TAP|YAP|RAP|PAP|WAD)\d)|UHID)\d{6}([A-J]|\d{1})\d{4}/0[1-6]$'
synth_claim_pattern = r'^(GAN_GENERATED)\d{7}([A-J]|\d{1})\d{4}/0[1-6]$'
synthetic_data['Claim No'] = synthetic_data['Claim No'].apply(lambda x: exrex.getone(synth_claim_pattern))

print(f'\nDuplicate rows in Synthetic Data = {synthetic_data.duplicated().sum()}')
synthetic_data.head(4)

Sampling rows: 100%|██████████| 1000/1000 [00:00<00:00, 4072.21it/s]



Duplicate rows in Synthetic Data = 0


,Claim No,Card No,Diagnosis,Surgery/Therapy,Provider Name,Date of Admit,Status,Status Date,Est Amt,Pre Auth Appr Amt,Surgery/Therapy Date,Discharged
0,GAN_GENERATED9308942A1031/02,APCMCO/ANT/2022/1/05042666,STEMI WITH ANGIO,Management Of Acute MI With Angiogram,SAVEERA INSTITUTE OF MEDICAL SCIENCES PVT LTD,2022-06-22,Discharge Update,2022-09-12,30299.0,30264.0,2022-07-03,2022-09-01
1,GAN_GENERATED1226048E2601/06,APTRUST/ANT/2022/2/00159126/02,DCMP,Medical Management of Acute Mi -Conservative M...,SAVEERA INSTITUTE OF MEDICAL SCIENCES PVT LTD,2022-04-13,Claim Paid,2022-06-21,30601.0,11377.0,2022-04-04,2022-04-15
2,GAN_GENERATED5756227H8487/04,APCMCO/ANT/2022/2/57174137,CAD ACS,Management Of Acute MI With Angiogram,TEJA NURSING HOME,2022-08-20,Discharge Update,2022-09-15,30646.0,30744.0,2022-08-21,2022-08-25
3,GAN_GENERATED8663254H5658/06,APCMCO/ANT/2022/2/38317604,CHEST PAIN,Medical Management of Acute Mi -Conservative M...,CHANDRA SUPERSPECIALTY HOSPITAL,2022-04-01,Claim Paid,2022-06-22,9721.0,5058.0,2022-04-06,2022-04-07


# Evaluation

## Quality Report

In [31]:
from sdv.evaluation.single_table import evaluate_quality

Qreport = evaluate_quality(
    real_data=df,
    synthetic_data=synthetic_data,
    metadata=metadata,
    verbose=True)

Qreport.get_visualization(property_name='Column Pair Trends')

Creating report: 100%|██████████| 4/4 [00:01<00:00,  2.01it/s]



Overall Quality Score: 73.35%

Properties:
Column Shapes: 79.2%
Column Pair Trends: 67.49%


## Diagnostic

In [32]:
from sdmetrics.reports.single_table import DiagnosticReport

Dreport = DiagnosticReport()
Dreport.generate(df, synthetic_data, metadata)

Creating report: 100%|██████████| 4/4 [00:09<00:00,  2.35s/it]


DiagnosticResults:

SUCCESS:
✓ The synthetic data covers over 90% of the numerical ranges present in the real data
✓ Over 90% of the synthetic rows are not copies of the real data
✓ The synthetic data follows over 90% of the min/max boundaries set by the real data

! The synthetic data is missing more than 10% of the categories present in the real data


In [33]:
Dreport.get_details(property_name='Coverage')

,Column,Metric,Diagnostic Score
0,Date of Admit,RangeCoverage,1.000000
1,Status Date,RangeCoverage,1.000000
2,Est Amt,RangeCoverage,1.000000
3,Pre Auth Appr Amt,RangeCoverage,0.998325
4,Surgery/Therapy Date,RangeCoverage,1.000000
5,Discharged,RangeCoverage,1.000000
6,Card No,CategoryCoverage,0.618031
7,Diagnosis,CategoryCoverage,0.803419
8,Surgery/Therapy,CategoryCoverage,0.933333
9,Provider Name,CategoryCoverage,0.896552


In [34]:
fig = Dreport.get_visualization(property_name='Coverage')
fig.show()

In [35]:
Dreport.get_details(property_name='Synthesis')

,Metric,Diagnostic Score,Num Matched Rows,Num New Rows
0,NewRowSynthesis,1.0,0,851


In [36]:
fig = Dreport.get_visualization(property_name='Synthesis')
fig.show()

In [37]:
Dreport.get_details(property_name='Boundaries')

,Column,Metric,Diagnostic Score
0,Date of Admit,BoundaryAdherence,0.968000
1,Status Date,BoundaryAdherence,0.918000
2,Est Amt,BoundaryAdherence,1.000000
3,Pre Auth Appr Amt,BoundaryAdherence,1.000000
4,Surgery/Therapy Date,BoundaryAdherence,0.973000
5,Discharged,BoundaryAdherence,0.986542


In [38]:
fig = Dreport.get_visualization(property_name='Boundaries')
fig.show()

## Feature Distributions

In [39]:
import matplotlib.pyplot as plt
from sdv.evaluation.single_table import get_column_plot

# fig, ax = plt.subplots(nrows= len(df.columns))

i=0;
for col in df.columns.drop(['Claim No']):
    fig = get_column_plot(
        real_data=df,
        synthetic_data=synthetic_data,
        column_name=col,
        metadata=metadata
    )
    fig.show()

In [40]:
# COlumn pair
from sdv.evaluation.single_table import get_column_pair_plot

fig = get_column_pair_plot(
    real_data=df,
    synthetic_data=synthetic_data,
    column_names=['Est Amt', 'Pre Auth Appr Amt'],
    metadata=metadata)
    
fig.show()

# Save the synthetic data

In [45]:
synthetic_data.to_csv('./synthetic_data_final.csv')

# Saving and Loading Synthesizer

In [46]:
synthesizer.save(
    filepath='./CTGANsynthesizer_final.pkl'
)

In [43]:
synthesizer = CTGANSynthesizer.load(
    filepath='./CTGANsynthesizer_final.pkl'
)

# Generating synthetic data from loaded model

In [44]:
regeneration - block to generate error.

SyntaxError: ignored

In [ ]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer

In [ ]:
synthetic_data2 = synthesizer.sample(num_rows=5000)
print(f'\nDuplicate rows in Synthetic Data = {synthetic_data2.duplicated().sum()}')
synthetic_data2.head(4)

## Saving the csv

In [ ]:
synthetic_data2.to_csv('./synthetic_data_5000_rows_15000_epochs_removed_computables.csv')